<a href="https://colab.research.google.com/github/fur-elyse/Cartoonization-AI/blob/master/data_collection/news/colab_tara_upload_manually_sourced_news_data_from_fb_berna_11232020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
!pip install python-docx

     |████████████████████████████████| 5.5MB 13.2MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=4f5b8a095d72ea42eabd77fa22dc53843dff2a81052e53fc3927bc52c12b8005
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


## Import Libraries

In [ ]:
import os
import uuid
import requests
import json
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
import docx

# EDIT ME/ CONFIG FILE

In [ ]:
# if you are uploading current news, set this value as 1.
# if you are uploading backlog news, set this value as 0.
UPDATE_CURR_NEWS = 1
 
#start date and end dates of news to be uploaded
START_DATE = '01252021'
END_DATE = '01262021'
 
#Connections
 
#Drive
 
# Change these according to your filepaths
 
KEYS_FP = "drive/MyDrive/DOT App/Data/Keys"
DOT_FP = "drive/MyDrive/DOT App"
DOT_APP_FP = "drive/MyDrive/Projects/DOT App/Data/News"
 
# Change this according to the month of the news you will be uploading
DOT_APP_NEWS_FP = "drive/MyDrive/DOT App/Data/News/2021/January"

# Connections (Drive, Firestore, GCS)

## Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
 
#edit this according to where your files are
 
#make this editable
#keys_fp = os.path.join(os.getcwd(),"drive","My Drive", "DOT App", "Data", "Keys")
#dot_fp = os.path.join(os.getcwd(),"drive","My Drive", "DOT App")
#dot_app_fp = os.path.join(os.getcwd(),"drive","My Drive", "DOT App", "Data", "News")
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive", "DOT App", "Data", "News", "2021", "January")
 
keys_fp = os.path.join(os.getcwd(), KEYS_FP)
dot_fp = os.path.join(os.getcwd(), DOT_FP)
dot_app_fp = os.path.join(os.getcwd(), DOT_APP_FP)
dot_app_news_fp = os.path.join(os.getcwd(), DOT_APP_NEWS_FP)

#January 2021 folder
dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "2021", "January")
 
#December folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "December")
 
#November folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "November")
 
#October folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "October")
 
#September folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "September")
 
#August folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "August")
 
#July folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "July")
 
#June folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "June")
 
#May folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "May")
 
#April folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "April")
 
#March folder
#dot_app_news_fp = os.path.join(os.getcwd(),"drive","My Drive","DOT App", "Data", "News", "March")
 
 
#/content/drive/My Drive/DOT App/Data/News/2021/January

Mounted at /content/drive


In [ ]:
print(keys_fp)

/content/drive/MyDrive/DOT App/Data/Keys


In [ ]:
import glob
print(glob.glob(dot_app_news_fp+"/*.jpg"))

[]


## Connect to GCS

In [ ]:
from google.colab import auth
auth.authenticate_user()
 
project_id = 'tara-prod-290208'
bucket_name = 'tara-images'
 
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
from google.cloud import storage
gcs_client = storage.Client.from_service_account_json(os.path.join(keys_fp, "dot_gcs_api.json"))

### Read Filenames from GCS

In [ ]:
#GCS filepath
# prefix = 'merchant_images/La Union/San Juan'
prefix = 'news_images/'
# gcs_filenames = [blob.id.split("/")[4].rsplit(".", 1)[0] for blob in gcs_client.list_blobs(bucket_name, prefix=prefix)]
gcs_filenames = [blob for blob in gcs_client.list_blobs(bucket_name, prefix=prefix)]

# gcs_filenames = [blob.id.split("/")[3].rsplit(".", 1)[0] for blob in gcs_client.list_blobs(bucket_name, prefix=prefix)]
gcs_filenames[:-19]

[<Blob: tara-images, news_images/, 1603116715121591>,
 <Blob: tara-images, news_images/01012021/01012021_Active COVID Cases_La Union.jpg, 1611538103177048>,
 <Blob: tara-images, news_images/01012021/01012021_COVID Case Bulletin_La Union.jpg, 1611538103161518>,
 <Blob: tara-images, news_images/01012021/01012021_COVID Case Profile_La Union.jpg, 1611538103204166>,
 <Blob: tara-images, news_images/01012021/01012021_COVID Heat Map_La Union.jpg, 1611538103294162>,
 <Blob: tara-images, news_images/01012021/01012021_COVID New Cases_La Union.jpg, 1611538103208474>,
 <Blob: tara-images, news_images/01012021/01012021_Description_La Union.docx, 1611538103215070>,
 <Blob: tara-images, news_images/01022021/01022021_Active COVID Cases_La Union.jpg, 1611538103780224>,
 <Blob: tara-images, news_images/01022021/01022021_COVID Case Bulletin_La Union.jpg, 1611538103304837>,
 <Blob: tara-images, news_images/01022021/01022021_COVID Case Profile_La Union.jpg, 1611538103742321>,
 <Blob: tara-images, news_imag

In [ ]:
# Check images of all things inside the bucket
# !gsutil ls -r gs://tara-images/**

### Upload files from Drive to GCS

#### EDIT ME

In [ ]:
# !gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/High\ Quality\ Merchant\ Photos/* gs://{bucket_name}/merchant_images/

# Make this editable

# Uploads the news images from the GDrive to the GCS

#January 2021
!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/2021/January/* gs://{bucket_name}/news_images/

#December
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/December/* gs://{bucket_name}/news_images/

#November
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/November/* gs://{bucket_name}/news_images/

#October
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/October/* gs://{bucket_name}/news_images/

#September
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/September/* gs://{bucket_name}/news_images/

#August
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/August/* gs://{bucket_name}/news_images/

#July
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/July/* gs://{bucket_name}/news_images/

#June
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/June/* gs://{bucket_name}/news_images/

#May
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/May/* gs://{bucket_name}/news_images/

#April
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/April/* gs://{bucket_name}/news_images/

#March
#!gsutil -m cp -r /content/drive/My\ Drive/DOT\ App/Data/News/March/* gs://{bucket_name}/news_images/

# Edit the filepath and uncomment this command to be able to upload the news images to GCS


Copying file:///content/drive/My Drive/DOT App/Data/News/2021/January/01012021/01012021_Description_La Union.docx [Content-Type=application/vnd.openxmlformats-officedocument.wordprocessingml.document]...
Copying file:///content/drive/My Drive/DOT App/Data/News/2021/January/01012021/01012021_Active COVID Cases_La Union.jpg [Content-Type=image/jpeg]...
Copying file:///content/drive/My Drive/DOT App/Data/News/2021/January/01022021/01022021_Description_La Union.docx [Content-Type=application/vnd.openxmlformats-officedocument.wordprocessingml.document]...
Copying file:///content/drive/My Drive/DOT App/Data/News/2021/January/01012021/01012021_COVID New Cases_La Union.jpg [Content-Type=image/jpeg]...
Copying file:///content/drive/My Drive/DOT App/Data/News/2021/January/01012021/01012021_COVID Case Bulletin_La Union.jpg [Content-Type=image/jpeg]...
Copying file:///content/drive/My Drive/DOT App/Data/News/2021/January/01012021/01012021_COVID Case Profile_La Union.jpg [Content-Type=image/jpeg]..

### Make Bucket Public

In [ ]:
# !gsutil iam ch allUsers:objectViewer gs://tara-data-bucket

## Connect to Firestore

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [ ]:
#Make this editable
#if not firebase_admin._apps["test"]:
   # test_creds = credentials.Certificate(os.path.join(os.getcwd(), "drive", "My Drive", "DOT App", "Data", "Keys", "dot_firestoretestdbaccess.json"))
    #test_db = firestore.client(firebase_admin.initialize_app(test_creds, name="test"))
#if not firebase_admin._apps["prod"]:
   # prod_creds = credentials.Certificate(os.path.join(os.getcwd(), "drive", "My Drive", "DOT App", "Data", "Keys", "dot_firestoreproddbaccess.json"))
   # prod_db = firestore.client(firebase_admin.initialize_app(prod_creds, name="prod"))

test_creds = credentials.Certificate(os.path.join(os.getcwd(), "drive", "My Drive", "DOT App", "Data", "Keys", "dot_firestoretestdbaccess.json"))
prod_creds = credentials.Certificate(os.path.join(os.getcwd(), "drive", "My Drive", "DOT App", "Data", "Keys", "dot_firestoreproddbaccess.json"))
 
#test_creds = credentials.Certificate(os.path.join(keys_fp, "dot_firestoretestdbaccess.json"))
#prod_creds = credentials.Certificate(os.path.join(keys_fp, "dot_firestoreproddbaccess.json"))
 
test_db = firestore.client(firebase_admin.initialize_app(test_creds, name="test"))
prod_db = firestore.client(firebase_admin.initialize_app(prod_creds, name="prod"))

## Connect to GSheets

In [ ]:
### to be implemented
### write df to gsheets directly instead of dling cv then uploading to drive

## Import colab notebook [WIP]

In [ ]:
# !pip install kora -q
# from kora import drive
# drive.link_nbs()

In [ ]:
# notebooks_fp = os.path.join(os.getcwd(), "drive", "My Drive", "DOT", "notebooks", "data_flow_and_cleaning.ipynb")

In [ ]:
# !cp drive/My\ Drive/DOT/notebooks/data_flow_and_cleaning .

In [ ]:
# import data_flow_and_cleaning

# Functions

## Get data from Firestore

In [ ]:
def get_data_from_firestore(db, collection_name):
    """
    Description:
    Load firestore data as a dataframe.
    ---
    Parameters:
    collection_name (str) - input the name of the collection in the firestore.
    ---
    Returns:
    dataframe of the data inside the collection.
    """
    docs = db.collection(collection_name)
    data = [doc.to_dict() for doc in docs.stream()]
    doc_id = [doc.id for doc in docs.stream()]
    
    df = pd.DataFrame(data)
    df["doc_id"] = doc_id
    return df

In [ ]:
# blogsdf_1020 = get_data_from_firestore(test_db, "blogs")

In [ ]:
# blogsdf_1020.to_csv("blogsdf_1020.csv", index=False)

## Delete Firestore Collection

In [ ]:
def delete_collection(db, collection_name, batch_size):
    """
    Description:
    Delete all documents in the collection.
    ---
    Parameters:
    db (str) - specify the firestore database you want to delete from.
    collection_name (str) - specify the firestore collection name you want to delete from.
    batch_size (int) - specify the size of the collection. You can also put an extremely large number, and it will stop once all files are deleted.
    """
    deleted = 0
    docs = db.collection(collection_name)
    snapshot_docs = [snapshot for snapshot in docs.limit(batch_size).stream()]

    for doc in tqdm(snapshot_docs):
        try:
            doc.reference.delete()
            deleted = deleted + 1
        except:
            continue

    if deleted >= batch_size:
        return delete_collection(docs, batch_size)

## Delete field in Firestore

In [ ]:
def delete_field(db, collection_name, document_name, field):
    """
    Description:
    Delete a specific field from a document.
    ---
    Parameters:
    db (str) - specify the firestore database you want to delete from.
    collection_name (str) - specify the firestore collection name you want to delete from.
    document_name (str) - specify the document name the field will be deleted from.
    field (str) - specify the field that will be deleted.
    """
    docs = db.collection(collection_name).document(document_name)
    docs.update({
        field: firestore.DELETE_FIELD
    })

## Upload data to Firestore

In [ ]:
# load missing data to firestore
# Convert sample df to json records
def upload_data_to_firestore(db, collection_name, document_names, df_to_upload):
    """
    Description:
    Upload your data to firestore.
    ---
    Parameters:
    db (str) - specify the firestore database you want to upload to.
    df_to_upload (df) - input the dataframe which will be converted into a dictionary for uploading.
    collection_name (str) - specify the firestore collection name you want to upload to. If collection name doesn't exist, it will make a new collection.
    document_names (int) - input the firestore document names you want to use.
    """
    df_to_upload_to_firestore = df_to_upload.to_dict('records')
    
    # create a collection
    docs = db.collection(collection_name)

    # Transfer data with Firestore's auto-hash
    [docs.document(document_names[i]).set(df_to_upload_to_firestore[i]) for i in tqdm(range(len(df_to_upload_to_firestore)))]

## Update field in Firestore

In [ ]:
def update_field(db, collection_name, document_name, update_dict):
    """
    Description:
    Update the fields in your firestore noSQL database.
    ---
    Parameters:
    db (str) - specify the firestore database you want to update.
    collection_name (str) - specify the firestore collection name you want to update.
    document_name (int) - input the firestore document name you want to update.
    update_dict (dict) - place the dictionary where key=field name and value=new value.
    """
    # open the document
    doc_ref = db.collection(collection_name).document(document_name)

    # place the field you want to update (dict)
    doc_ref.update(update_dict)
    # return document_name

## Firestore Query and retrieves docs from Firestore (returns doc_ids)

In [ ]:
def retrieve_docs_from_firestore(db, collection_name, q_field, q_operator, q_value):
    """
    Description:
    Queries data from firestore.
    ---
    Parameters:
    collection_name (str) - input the name of the collection in the firestore.
    q_field (str) - input the field name of the query filter in the firestore.
    q_operator (str) - input the operator of the query filter in the firestore.
    q_value (str) - input the value of the query filter in the firestore.

    ---
    Returns:
    doc_ids (list) - list of doc_ids retrieved from the corresponding query
    """

    docs = db.collection(collection_name).where(q_field, q_operator, q_value).stream()
    doc_ids = [doc.id for doc in docs]
    # for doc in docs:
        # print(f'{doc.id} => {doc.to_dict()}')

    return doc_ids

## Generate UUID

In [ ]:
def generate_uuid(length):
    """
    Description:
    Generate a list of unique ids based on the length given.
    ---
    Parameters:
    length (int) - how many unique ids do you need to generate.
    ---
    Returns:
    A list of unique ids.
    """
    return [str(uuid.uuid4().hex) for x in range(length)]

In [ ]:
def tag_number(number, destination):

    if destination == "San Juan, La Union":
        # mobile numbers
        if (len(number) == 10) and (number[0]=="9"):
            return "mobile", "63" + number
        elif (len(number) == 11) and (number[:2]=="09"):
            return "mobile", "63" + number[1:]
        elif (len(number) == 12) and (number[:3]=="639"):
            return "mobile", number
        
        # telephone numbers
        elif (len(number) == 7):
            return "telephone", "6372" + number
        elif (len(number) == 9) and (number[:2]=="72"):
            return "telephone", "63" + number
        elif (len(number) == 10) and (number[:3]=="072"):
            return "telephone", "63" + number[1:]
        elif (len(number) == 11) and (number[:4]=="6372"):
            return "telephone", number
        
        # skip numbers
        else:
            return "skip", ""

In [ ]:
def jsonify_number(number_type, number, number_dict):
    if (number_type == "mobile") and (number != ""):
        if "mobile_number" not in number_dict:
            number_dict["mobile_number"] = {}
        number_dict["mobile_number"]["number_"+str(len(number_dict["mobile_number"])+1)] = number
    if (number_type == "telephone") and (number != ""):
        if "telephone_number" not in number_dict:
            number_dict["telephone_number"] = {}
        number_dict["telephone_number"]["number_"+str(len(number_dict["telephone_number"])+1)] = number

In [ ]:
def parse_numbers(numbers):
    number_dict = dict()
    [jsonify_number(tag_number(number, "San Juan, La Union")[0], tag_number(number, "San Juan, La Union")[1], number_dict) for number in numbers]
    return number_dict

In [ ]:
def clean_data_for_firestore(df):
    # add missing columns
    for col in test_df.columns.difference(df.columns):
        df[col] = np.nan

    # fill in missing data
    # fill the empty cells as "0" which means they were not part of DOT's list
    df["dotaccredited"] = df["dotaccredited"].fillna("0").apply(str)
    # fill the empty cells as 0s which means they have never been rated before
    df["rating"] = df["rating"].fillna(0).apply(int)
    # the rest of the missing empty will have empty strings so it can be parsed in the app
    df = df.fillna("")

    # clean values
    # clean phone numbers by splitting the string into a list and parse them into a json format
    df["phone"] = df[["phone"]].applymap(lambda x: re.sub(r'/',',', x) if isinstance(x, str) else x)
    df["phone"] = df[["phone"]].applymap(lambda x: re.sub(r'[^\d,]+','', str(x)).split(",") if not isinstance(x, dict) else x)
    df["phone"] = df[["phone"]].applymap(lambda x: parse_numbers(x) if not isinstance(x, dict) else x)
    # rem
    df["address"] = df["address"].apply(lambda x: re.sub(r'[^a-zA-Z0-9 .,]+','', str(x)))
    df["email"] = df["email"].apply(lambda x: re.sub(r'[^a-z0-9_@.]+','', str(x.lower())))
    df['services'] = df[['services']].applymap(lambda x: {} if len(str(x))==0 else ast.literal_eval(x) if not isinstance(x, dict) else x)
    df['amenities'] = df[['amenities']].applymap(lambda x: [] if len(str(x))==0 else ast.literal_eval(x) if not isinstance(x, list) else x)
    df['products'] = df[['products']].applymap(lambda x: [] if len(str(x))==0 else ast.literal_eval(x) if not isinstance(x, list) else x)
    df["lat"] = df["lat"].apply(str)
    df["lng"] = df["lng"].apply(str)
    df["business_hours"] = df["business_hours"].str.replace("â€“","-").str.replace("'",'"').str.replace("True","true").str.replace("False","false")
    df = df[list(test_df.columns)]
    return df

# News Data Pipeline

In [ ]:
from os import walk

mypath = dot_app_news_fp

start_date = START_DATE
end_date = END_DATE

f = []

for (dirpath, dirnames, filenames) in walk(mypath):
    included = 0
    if len(filenames)>0:
        news_date = filenames[0].split('_')[0]
    
        if news_date >= start_date and news_date <= end_date:
            # print("pumasok sa news")
            included = 1
    if included == 1:
        f.extend(filenames)
    
        # news_date = filenames[0]
        # print(news_date)
        # print(type(news_date))
        

        # print(filenames[0])
        # print(type(filenames))

    # break

In [ ]:
for fname in f:
    print(fname)

01252021_Description_La Union.docx
01252021_COVID Case Bulletin_La Union.jpg
01252021_COVID New Cases_La Union.jpg
01252021_COVID Heat Map_La Union.jpg
01252021_COVID Case Profile_La Union.jpg
01252021_Active COVID Cases_La Union.jpg
01262021_Description_La Union.docx
01262021_COVID Case Bulletin_La Union.jpg
01262021_COVID New Cases_La Union.jpg
01262021_COVID Heat Map_La Union.jpg
01262021_COVID Case Profile_La Union.jpg
01262021_Active COVID Cases_La Union.jpg


In [ ]:
def read_docx(docx_name):
    doc = docx.Document(docx_name)
 
    full_text = ""
    for paragraph in doc.paragraphs:
        # full_text.append(paragraph.text)
        full_text = full_text + paragraph.text


    # add 2 lines between paragraphs and merge them all
    "\n\n".join(full_text)
    # print(f'full_text: {full_text}')

    return full_text

In [ ]:
def format_timestamp(timestamp):
    """ Formats timestamp with the format YYYYMMDDHHmmss to a date format DD Month Name Year """
    # timestamp = '20200807232822'
    timestamp = timestamp[:8]
    # Create date object in given time format yyyy-mm-dd
    my_date = datetime.strptime(timestamp, "%Y%m%d")
    format = "%d %B %Y"
    #format datetime using strftime()
    formatted_my_date = my_date.strftime(format)
    # print(my_date)
    # print('type: ',type(my_date))
    # print("formatted datetime:", formatted_my_date)
    return formatted_my_date

### Trial function

In [ ]:
# from os import walk

# mypath = dot_app_news_fp

# start_date = START_DATE
# end_date = END_DATE

# f = []

# for (dirpath, dirnames, filenames) in walk(mypath):
#     included = 0
#     if len(filenames)>0:
#         news_date = filenames[0].split('_')[0]
    
#         if news_date >= start_date and news_date <= end_date:
#             print("pumasok sa news")
#             included = 1
#     if included == 1:
#         f.extend(filenames)

### News Parsing Function

#### Update Previous Highlighted News

In [ ]:
if UPDATE_CURR_NEWS == 1:
    # Get previous highlighted news
    highlighted_news_test = retrieve_docs_from_firestore(test_db, "blogs", "highlight", "==", "1")
    highlighted_news_prod = retrieve_docs_from_firestore(prod_db, "blogs", "highlight", "==", "1")

    # #Update previous highlighted news
    if len(highlighted_news_test) > 0:
        update_field(test_db, "blogs", highlighted_news_test[0], {"highlight":""})
    if len(highlighted_news_prod) > 0:
        update_field(prod_db, "blogs", highlighted_news_prod[0], {"highlight":""})

#### Upload latest news

In [ ]:
data = []

directory = dot_app_news_fp

# #returns the directories
subdir = [x[0] for x in os.walk(directory)]
subdir.pop(0)

start_date = START_DATE
end_date = END_DATE

for dir in subdir:
    s = []
    print("\n\n==========") 
    print(dir)
    #folder_name == news_date
    folder_name = dir.split("/")[-1]
    # print(folder_name)
    # news_date == folder_name
    
    print("news_date: ", folder_name)
    if folder_name >= start_date and news_date <= end_date:
        print("pumasok")
        is_latest_news_entry = ""
        latest_news_entry = end_date
        if UPDATE_CURR_NEWS == 1:
            if folder_name == latest_news_entry:
                is_latest_news_entry = "1"
        # # timestamp correct format in the app
        # # 20200807232822
        # # YYYYMMDDHHMMSS

        timestamp = f'{folder_name[-4:]}{folder_name[:4]}000000'
        # print(timestamp)

        # gets all files in a folder/subdir
        f_list =[x[2] for x in os.walk(dir)]
        s.extend(f_list)

        for files in s:
            
            print(files)
            print("\n*****")
            
            description = ""
            for fname in files:
                print("==")
                print(fname)
                fname_url = fname.replace(" ", "%20")
                # print(fname_url)

                f_ext = fname.split(".")[1]
                print(f'f_ext: {f_ext}')

                if f_ext == "jpg":
                    img_desc = f'<br><br> <img src="https://storage.googleapis.com/tara-images/news_images/{folder_name}/{fname_url}" alt="{fname}"> <br><br>'
                    description = description + img_desc
                
                if f_ext == "docx":
                    parsed_doc = read_docx(dir+"/"+fname)
                    # ln_break = " <br> <br> "
                    # parsed_doc = "".join((parsed_doc, ln_break))
                    description = "".join((parsed_doc, description)) 

                fname_date = fname.split("_")[0]
                # print(fname_date)

                # fname_parts = fname.split("_")
                # print(f_ext)
                # print(fname_parts)
            print(f'description: {description}')

        # tara-images/news_images/10012020/10012020_Active COVID Cases_La Union.jpg
        # https://storage.googleapis.com/tara-images/news_images/10012020/10012020_Active%20COVID%20Cases_La%20Union.jpg
        # gcs_url = f"https://storage.googleapis.com/tara-images/news_images/10012020/10012020_Active%20COVID%20Cases_La%20Union.jpg"
        # gcs_url_sample = f"https://storage.googleapis.com/tara-images/news_images/{10012020}/{10012020_Active%20COVID%20Cases_La%20Union.jpg}"

        # Adding the COVID-19 Monitoring Dashboard Photo
        files.append("COVID-19 Monitoring Dashboard_La Union.jpg")
        #get a random image cover in files:
        while True:       
            selected_img = random.choice(files)
            if selected_img.split(".")[1] == "jpg":
                img_cover = selected_img.replace(" ", "%20")
                break

        formatted_date = format_timestamp(timestamp)
        src_url = "https://www.facebook.com/LGUlaunion"
        src_name = "Provincial Government of La Union"
        title = f'{formatted_date} La Union COVID-19 Monitoring Updates'

        if img_cover == "COVID-19%20Monitoring%20Dashboard_La%20Union.jpg":
            img_url = f'https://storage.googleapis.com/tara-images/news_images/{img_cover}'
        else:
            img_url = f'https://storage.googleapis.com/tara-images/news_images/{folder_name}/{img_cover}'

        loves = 0
            
        row_data = {
            "timestamp" : timestamp,
            "source name" : src_name,
            "title" : title,
            "highlight" : is_latest_news_entry,
            "image url" : img_url, 
            "loves" : loves,
            "source" : src_url,
            "description": description,
            "date" : formatted_date,
            "doc_id" : generate_uuid(1)[0]
            }
        
        data.append(row_data)
        
df = pd.DataFrame.from_dict(data)



/content/drive/My Drive/DOT App/Data/News/2021/January/01022021
news_date:  01022021


/content/drive/My Drive/DOT App/Data/News/2021/January/01012021
news_date:  01012021


/content/drive/My Drive/DOT App/Data/News/2021/January/01052021
news_date:  01052021


/content/drive/My Drive/DOT App/Data/News/2021/January/01042021
news_date:  01042021


/content/drive/My Drive/DOT App/Data/News/2021/January/01062021
news_date:  01062021


/content/drive/My Drive/DOT App/Data/News/2021/January/01072021
news_date:  01072021


/content/drive/My Drive/DOT App/Data/News/2021/January/01082021
news_date:  01082021


/content/drive/My Drive/DOT App/Data/News/2021/January/01092021
news_date:  01092021


/content/drive/My Drive/DOT App/Data/News/2021/January/01102021
news_date:  01102021


/content/drive/My Drive/DOT App/Data/News/2021/January/01032021
news_date:  01032021


/content/drive/My Drive/DOT App/Data/News/2021/January/01112021
news_date:  01112021


/content/drive/My Drive/DOT App/Data/News

In [ ]:
df

,timestamp,source name,title,highlight,image url,loves,source,description,date,doc_id
0,20210125000000,Provincial Government of La Union,25 January 2021 La Union COVID-19 Monitoring U...,,https://storage.googleapis.com/tara-images/new...,0,https://www.facebook.com/LGUlaunion,"Kaprobinsiaan, here is the La Union CoViD-19 C...",25 January 2021,098e93e34b974db7afb71aaeca238fef
1,20210126000000,Provincial Government of La Union,26 January 2021 La Union COVID-19 Monitoring U...,1,https://storage.googleapis.com/tara-images/new...,0,https://www.facebook.com/LGUlaunion,"Kaprobinsiaan, here is the La Union CoViD-19 C...",26 January 2021,5d4979b900e94d50bf8d08fb08ac560e


## Download CSV

In [ ]:
# df.to_csv("news.csv", index=False)

## Upload News from GDrive to Firestore

In [ ]:
#uploads to test_db
upload_data_to_firestore(test_db, "blogs", df.doc_id.to_list(), df)

#uploads to prod_db
upload_data_to_firestore(prod_db, "blogs", df.doc_id.to_list(), df)

100%|██████████| 2/2 [00:00<00:00,  3.76it/s]
